## Things to all in this md

- [ ] Calling Comparison APIs
- [ ] Test Set Data to compare apis
- [ ] Select metrics that should me used to compare APIs
- [ ] Create list of metrics and visualizatinos needed to make this a nice presentable Notebook.

In [1]:
from corpus_based.sent_sim import CorpusBasedComparision
from sent_bert.sent_sim_bert import MultiSentenceBertComparision

In [2]:
CorpusBasedComparision("nice day", "bad dog")

0.20143834189646437

In [3]:
MultiSentenceBertComparision(["Nice Day"],["Nice time","good time","Nice day","bad dog"])

tensor([[0.7998, 0.6919, 1.0000, 0.1352]])

In [13]:
from data.read_tsv import read_file

In [41]:
parsed_data = read_file("./data/sick_data.tsv")

In [44]:
for i in range(1,10):
    s1, s2, ex = parsed_data[i][1],parsed_data[i][2], parsed_data[i][0]
    print( ex , 
          CorpusBasedComparision(s1,s2)*5 ,
          float(MultiSentenceBertComparision([s1],[s2])[0][0])*5,
          s1, s2, "" , sep="\n")
    print()
    

4.4
4.690371912294329
4.524145424365997
SC dismisses govt's review plea in Vodafone tax case
SC dismisses govt's review petition on Vodafone tax verdict


2.6
2.825589797309209
3.379615545272827
Explosion hits oil pipeline in Syria's Homs
Explosion hits pipeline as Assad attacks cities


3.8
2.4418354596831606
4.573043882846832
NATO Soldier Killed In Afghan Attack
NATO soldier killed in S. Afghanistan


4.2
3.879978199630138
4.657001197338104
Castro celebrates 86th birthday Monday
Fidel Castro celebrates 86th birthday


3
2.2563983924499404
2.904941439628601
Israel ex-spy warns against 'messianic' Iran war
Israeli ex-spy boss: no trust in leadership over Iran – video


3.8
4.3865897286288185
3.937733769416809
Eye Opener: Supreme Court rules on Arizona immigration law
U.S. Supreme Court issues split ruling on Arizona immigration law


3.2
3.3313505594046386
3.0571824312210083
NATO traces path out of Afghanistan
NATO Leaders Commit to Afghanistan Transition


4
3.1070439955801836
4.46950

## list

- accuacy, precision
- error
- deviation
- r2 error
- FP-TN ( DA COURSE LAST SLIDES )
- biases ( corpus stats (word order ) )